In [3]:
import numpy as np
import pandas as pd

# import nltk
# from nltk.tokenize import word_tokenize
# nltk.download('punkt_tab')

from sklearn.feature_extraction.text import CountVectorizer

In [4]:
df = pd.read_csv("../dataset/final_dataset/sentiment_analysis_dataset.csv")
df.head()

,text,label
0,matt forte steve smith cin,negative
1,but not possible others nice try modi what goo...,positive
2,let complete visionary mms that denied this an...,positive
3,dont see much longevity for govts rajasthan ka...,positive
4,pappu may ask evidence for this too jai modi,negative


In [5]:
df["label"] = df["label"].replace({
    "negative":0,
    "positive":1
})

C:\Users\Saurav\AppData\Local\Temp\ipykernel_13464\2866728110.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["label"] = df["label"].replace({


In [6]:
df.head()

,text,label
0,matt forte steve smith cin,0
1,but not possible others nice try modi what goo...,1
2,let complete visionary mms that denied this an...,1
3,dont see much longevity for govts rajasthan ka...,1
4,pappu may ask evidence for this too jai modi,0


# Data Splitting

In [7]:
X = df["text"]
y = df["label"]

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=41)

# Text Vectorization

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [78]:
# binary is true because we want to know if certain hate words orccurs in a text data or not rather 
# than calculating the frequency.

tfidf = TfidfVectorizer(max_features=500)
X_train_trf = tfidf.fit_transform(X_train)
X_test_trf = tfidf.transform(X_test)

In [79]:
X_train_trf.toarray()

array([[0.10965012, 0.        , 0.        , ..., 0.05717945, 0.        ,
        0.08202022],
       [0.        , 0.        , 0.        , ..., 0.38512108, 0.        ,
        0.        ],
       [0.        , 0.        , 0.47978385, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.32342432, 0.        ,
        0.        ]])

# Model Building

In [80]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score,precision_score,recall_score,classification_report

In [81]:
nb = GaussianNB()
nb.fit(X_train_trf.toarray(),y_train)
y_pred = nb.predict(X_test_trf.toarray())


In [82]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.72      0.76      0.74      1067
           1       0.71      0.67      0.69       933

    accuracy                           0.72      2000
   macro avg       0.72      0.71      0.71      2000
weighted avg       0.72      0.72      0.72      2000



# Deep Learning

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,SimpleRNN,Flatten,Embedding, LSTM

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df["text"])

In [12]:
# count no of sentences or rows
tokenizer.document_count

263794

In [13]:
X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_test_sequences = tokenizer.texts_to_sequences(X_test)


In [14]:
X_train_padded_sequences = pad_sequences(sequences=X_train_sequences,padding="post",maxlen=50)
X_test_padded_sequences = pad_sequences(sequences=X_test_sequences,padding="post",maxlen=50)

In [15]:
X_train_padded_sequences.shape,X_test_padded_sequences.shape


((211035, 50), (52759, 50))

In [18]:
vocab_size = len(tokenizer.word_index) + 1
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=100))
model.add(LSTM(units=50, return_sequences=True))
model.add(LSTM(units=50))
model.add(Dense(units=2, activation='softmax'))
model.summary()

f:\Data Science and ML\End To End ML\Youtube Sentiment Analysis\Youtube-Comments-Sentiments-Analysis\venv-youtube-sentiment\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
model.fit(X_train_padded_sequences,y_train,epochs=20,validation_data=(X_test_padded_sequences,y_test))

Epoch 1/20
6595/6595 ━━━━━━━━━━━━━━━━━━━━ 926s 139ms/step - accuracy: 0.8110 - loss: 0.3954 - val_accuracy: 0.9245 - val_loss: 0.1833
Epoch 2/20
 100/6595 ━━━━━━━━━━━━━━━━━━━━ 15:56 147ms/step - accuracy: 0.9407 - loss: 0.1503

KeyboardInterrupt: 